In [5]:
import random as rand, numpy as np, os, sys
from collections import deque

In [6]:
class Node():
    def __init__(self, state, parent, action, distance_to):
        self.state = state
        self.parent = parent
        self.action = action
        if parent is None:
            self.distance_travelled = 0
        else:
            self.distance_travelled = self.parent.distance_travelled + 1
        self.distance_to = distance_to

In [7]:
class StackFrontier():
    def __init__(self):
        self.frontier = deque()
    
    def add(self, node:Node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)

    def pop(self):
        if self.empty():
            raise Exception('Frontier is empty')
        else:
            return self.frontier.popleft()


class QueueFrontier():
    def __init__(self):
        self.frontier = []
    
    def add(self, node:Node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)

    def pop(self):
        if self.empty():
            raise Exception('Frontier is empty')
        else:
            return self.frontier.pop(0)


class AStarFrontier():
    def __init__(self):
        self.frontier = []
    def add(self, node: Node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("Frontier is empty")
        else:
            minDist = sys.maxsize
            for i, node in enumerate(self.frontier):
                if node.distance_travelled + node.distance_to < minDist:
                    retIndex = i
                    minDist = node.distance_travelled + node.distance_to
            return self.frontier.pop(retIndex)
    

class GreedyFrontier():
    def __init__(self):
        self.frontier = []
    def add(self, node: Node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("Frontier is empty")
        else:
            minDist = 999999
            for i, node in enumerate(self.frontier):
                if node.distance_to < minDist:
                    retIndex = i
                    minDist = node.distance_to
            return self.frontier.pop(retIndex)


class LearnedFrontier():
    def __init__(self, bias = rand.randint(0,1), dtr_weight = rand.randint(0,1), dt_weight = rand.randint(0,1)):
        self.bias = bias
        self.dist_trav_weight = dtr_weight
        self.dist_to_weight = dt_weight

    def add(self, node:Node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)

    def pop(self):
        if self.empty():
            raise Exception('Frontier is empty')
        else:
            minDist = 999999
            for i, node in enumerate(self.frontier):
                if node.distance_to < minDist:
                    retIndex = i
                    minDist = self.bias + self.dist_to_weight * node.distance_to + self.dist_trav_weight * node.distance_travelled
            return self.frontier.pop(retIndex)


In [8]:
class Maze():
    def __init__(self, walls = [], start = None, goal = None):

        self.walls = walls # 2D bool array
        self.width = max(len(row) for row in walls) if walls != [] else 0
        self.height = len(walls)
        self.start = start
        self.goal = goal

    def read_maze(self, filename:str):
        # Read file
        with open(filename, 'r') as f:
            contents = f.read()
        
        if contents.count('A') != 1:
            raise Exception("Maze needs a single startpoint")
        if contents.count('B') != 1:
            raise Exception("Maze needs a single endpoint")

        contents = contents.splitlines()
        # Record height and width of maze
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Add context
        self.walls = [] # 2D bool array
        for r in range(self.height):
            row = []
            for c in range(self.width):
                # Try block as not all mazes will be rectangular
                try:
                    if contents[r][c] == '#':
                        row.append(True)
                    elif contents[r][c] == ' ':
                        row.append(False)
                    elif contents[r][c] == 'A':
                        self.start = (r,c)
                        row.append(False)
                    elif contents[r][c] == 'B':
                        self.goal = (r,c)
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(True)
            self.walls.append(row)
        self.solution = None

    def get_distance(self, state1:tuple, state2:tuple):
        return abs(state1[0] - state2[0]) + abs(state1[1] - state2[1])

    def write_maze(self, maze_name:str):
        new_maze = open(maze_name, 'w')
        new_maze.write('Dimensions: (' + str(self.height) + ',' + str(self.width) + ')\n')
        for r in range(self.height):
            for c in range(self.width):
                if self.walls[r][c] == True:
                    new_maze.write('#')
                elif (r,c) == self.start:
                    new_maze.write('A')
                elif (r,c) == self.goal:
                    new_maze.write('B')
                else:
                    new_maze.write(' ')

            new_maze.write('\n')
        new_maze.close()

        


In [9]:
class Agent():
    def __init__(self, frontier):
        self.current_state = None
        self.explored = set()
        self.frontier = frontier
    
    def neighbors(self, maze:Maze):
        row, col = self.current_state
        # All possible actions
        candidates = [
            ('up', (row-1, col)),
            ('down', (row+1, col)),
            ('left', (row, col-1)),
            ('right', (row, col+1)),
        ]
        # Validate actions
        possible = []
        for action, (row, col) in candidates:
            try:
                if not maze.walls[row][col]:
                    possible.append((action, (row, col)))
            except IndexError:
                continue
        return possible

    def solve(self, maze:Maze):
        # Keep track of how many states have been explored (performance measure)
        # self.num_explored = 0

        # Initialize frontier to the starting position
        start = Node(state=maze.start, parent=None, action=None, distance_to=maze.get_distance(maze.start, maze.goal))
        self.frontier.add(start)


        # Search for solution
        while True:
            if self.frontier.empty():
                # cprint('No solution', 'red', end='\n\n')
                # print('No Solution', end='')
                return False
            node = self.frontier.pop()
            self.current_state = node.state
            if self.current_state == maze.goal:
                # Add to explored
                self.explored.add(self.current_state)
                # cprint('Solved!', 'green', end='\n\n')
                # print('Solved!', end='')
                actions = []
                cells = []

                # Follow breadcrumbs to remake solution path
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                # B -> A --> A -> B
                actions.reverse()
                cells.reverse()
                maze.solution = (actions, cells)
                return len(cells)
            # Add to explored
            self.explored.add(self.current_state)

            # Add neighbors to frontier
            for action, state in self.neighbors(maze):
                if not self.frontier.contains_state(state):
                    if state not in self.explored: 
                        child = Node(state=state, parent=node, action=action, distance_to=maze.get_distance(state, maze.goal))
                        self.frontier.add(child)
            
    def print_solution(self, maze:Maze, fog_of_war:bool = False):
        for row in range(maze.height):
            for col in range(maze.width):
                if fog_of_war:
                    if (row,col) in self.explored:
                        if (row,col) == maze.start:
                            # cprint('A', 'cyan', end='')
                            print('A', end='')
                        elif (row,col) == maze.goal:
                            # cprint('B', 'cyan', end='')
                            print('B', end='')
                        elif maze.walls[row][col] == True:
                            print('#', end='')
                        elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                            # cprint('*', 'green', end='')
                            print('*', end='')
                        elif (row,col) in self.explored:
                            # cprint('e', 'red', end='')
                            print('e', end='')
                        else:
                            print(' ', end='')
                    else:
                        print('?', end='')
                else:
                    if (row,col) == maze.start:
                        # cprint('A', 'cyan', end='')
                        print('A', end='')
                    elif (row,col) == maze.goal:
                        # cprint('B', 'cyan', end='')
                        print('B', end='')
                    elif maze.walls[row][col] == True:
                        print('#', end='')
                    elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                        # cprint('*', 'green', end='')
                        print('*', end='')
                    elif (row,col) in self.explored:
                        # cprint('e', 'red', end='')
                        print('e', end='')
                    else:
                        print(' ', end='')
            print()
        print()

        actions, cells = maze.solution
        print('States explored: ' + str(len(self.explored)))
        print('Length of solution: ' + str(len(actions)))



In [10]:
class MazeGenerator():
    def __init__(self, maze:Maze, min_dimensions:tuple, max_dimensions:tuple, min_path_length:int, start_num:int=0):
        self.maze = maze
        self.min_height, self.min_width = min_dimensions
        self.max_height, self.max_width = max_dimensions
        self.min_path_length = min_path_length
        self.start_num = start_num

    def generate(self, num_maps):
        def feature_given_int(key:int):
            feature_dict = {
                0:' ',
                1:'#'
            }
            return feature_dict[key]
        
        def verify(maze_name:str):
            maze = InfoMaze(maze_name)
            solver = Agent(AStarFrontier())
            return solver.solve(maze)

        generated = self.start_num
        while generated < num_maps + self.start_num:
            map_name = 'gmap' + str(generated) + '.txt'
            height = rand.randint(self.min_height, self.max_height)
            width = rand.randint(self.min_width, self.max_width)
            start_pos = (rand.randint(0, height - 1), rand.randint(1, width))
            goal_pos = (rand.randint(0, height - 1), rand.randint(1, width))
            if goal_pos == start_pos:
                goal_pos = (rand.randint(0, height), rand.randint(1, width))
            print('Map specs: ' + map_name + '\n Dimensions=(' + str(height) + ',' + str(width) + ')\n')
            is_maze = False
            while is_maze == False:
                new_map = open(map_name, 'w')
                new_map.write('Dimensions: (' + str(height) + ',' + str(width) + ')\n')
                new_map.write('#' * (width + 2) + '\n')
                for r in range(height):
                    for c in range(width + 2):
                        if c == 0 or c == width + 1:
                            new_map.write('#')
                        elif (r,c) == start_pos:
                            new_map.write('A')
                        elif (r,c) == goal_pos:
                            new_map.write('B')
                        else:
                            new_map.write(feature_given_int(rand.randint(0,1)))
                    new_map.write('\n')
                new_map.write('#' * (width + 2) + '\n')
                new_map.close()
                test_result = verify(map_name)
                if test_result == False:
                    continue
                elif test_result < self.min_path_length:
                    continue
                else:
                    test_map = open(map_name, 'a')
                    test_map.write('Optimal path length: ' + str(test_result))
                    test_map.close()
                    print('Maze found')
                    is_maze = True
            generated += 1

In [11]:
class MazeGenerator2():
    def __init__(self, min_dimensions:tuple, max_dimensions:tuple, min_path_length:int, start_num:int=0):
        self.maze = None
        self.min_height, self.min_width = min_dimensions
        self.max_height, self.max_width = max_dimensions
        self.min_path_length = min_path_length
        self.start_num = start_num

    def generate(self, num_maps):
        def feature_given_int(key:int):
            feature_dict = {
                0:False,
                1:True
            }
            return feature_dict[key]
        
        def verify(generator:MazeGenerator2, bwalls:list, start:tuple, goal:tuple):
            generator.maze = Maze(bwalls, start, goal)
            solver = Agent(AStarFrontier())
            return solver.solve(generator.maze)

        generated = self.start_num
        while generated < num_maps + self.start_num:
            map_name = 'gmap' + str(generated) + '.txt'
            height = rand.randint(self.min_height, self.max_height)
            width = rand.randint(self.min_width, self.max_width)
            start_pos = (rand.randint(0, height), rand.randint(1, width))
            goal_pos = (rand.randint(0, height), rand.randint(1, width))
            if goal_pos == start_pos:
                goal_pos = (rand.randint(0, height), rand.randint(1, width))
            print(f'Map specs: {map_name}\n Dimensions=({height},{width})\n')
            is_maze = False
            creation_attempts = 0
            while is_maze == False:
                walls = []
                for r in range(height + 2):
                    row = []
                    for c in range(width + 2):
                        if c == 0 or c == width + 1 or r == 0 or r == height + 1:
                            row.append(True)
                        elif (r - 1,c) == start_pos:
                            row.append(False)
                        elif (r - 1,c) == goal_pos:
                            row.append(False)
                        else:
                            row.append(feature_given_int(rand.randint(0,1)))
                    walls.append(row)
                test_result = verify(self, bwalls=walls, start=start_pos, goal=goal_pos)
                creation_attempts += 1
                if test_result == False:
                    print(f'Generation {creation_attempts} was unsolvable')
                    continue
                elif test_result < self.min_path_length:
                    print(f'Generation {creation_attempts} had a solution length less than the minimum threshold')
                    continue
                else:
                    self.maze.write_maze(map_name)
                    test_map = open(map_name, 'a')
                    test_map.write(f'Optimal path length: {test_result}')
                    test_map.close()
                    print(f'Generation {creation_attempts} successfully made a maze')
                    is_maze = True
            generated += 1


In [12]:
mgen = MazeGenerator2((5,5), (10,10), 5, 254)
mgen.generate(25)

neration 48395 was unsolvable
Generation 48396 had a solution length less than the minimum threshold
Generation 48397 had a solution length less than the minimum threshold
Generation 48398 had a solution length less than the minimum threshold
Generation 48399 was unsolvable
Generation 48400 had a solution length less than the minimum threshold
Generation 48401 had a solution length less than the minimum threshold
Generation 48402 was unsolvable
Generation 48403 was unsolvable
Generation 48404 was unsolvable
Generation 48405 had a solution length less than the minimum threshold
Generation 48406 had a solution length less than the minimum threshold
Generation 48407 had a solution length less than the minimum threshold
Generation 48408 was unsolvable
Generation 48409 was unsolvable
Generation 48410 was unsolvable
Generation 48411 was unsolvable
Generation 48412 had a solution length less than the minimum threshold
Generation 48413 was unsolvable
Generation 48414 had a solution length less

In [10]:
#mlazer2 = Maze()
#mlazer2.read_file('maze3.txt')
#rational = Agent(GreedyFrontier())
#rational.solve(mlazer2)
#rational.print_solution(mlazer2, fog_of_war=False)

In [27]:
f = open('gmap0.txt')
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'gmap0.txt'

In [30]:
mgen.maze.write_maze('broke.txt')
print(mgen.maze.goal)
print(mgen.maze.walls)

(0, 3)
[[True, True, True, True, True, True, True, True, True, True, True, True], [True, False, False, False, False, True, False, False, False, False, True, True], [True, True, False, True, True, True, False, True, False, False, False, True], [True, True, True, False, True, False, True, False, False, False, False, True], [True, False, True, False, True, False, False, False, True, False, True, True], [True, False, False, False, False, False, True, True, True, False, False, True], [True, False, False, True, True, False, True, True, True, True, True, True], [True, True, True, True, True, True, True, True, True, True, True, True]]
